In [2]:
import yt_dlp as youtube_dl
import requests
import pytube
import re
import gradio as gr
from langchain_community.document_loaders import YoutubeLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
import tiktoken
from dotenv import load_dotenv

load_dotenv() ## load all the environment variables

import streamlit as st
import os
import google.generativeai as genai
from PIL import Image


2024-11-17 15:47:09,650 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:09,652 [DEBUG] load_verify_locations cafile='/opt/anaconda3/envs/aienv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-11-17 15:47:09,657 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:09,657 [DEBUG] load_verify_locations cafile='/opt/anaconda3/envs/aienv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-11-17 15:47:09,677 [DEBUG] connect_tcp.started host='api.gradio.app' port=443 local_address=None timeout=3 socket_options=None
2024-11-17 15:47:09,681 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:09,682 [DEBUG] load_verify_locations cafile='/opt/anaconda3/envs/aienv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-11-17 15:47:09,728 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:09,729 [DEBUG] load_verify_locations cafile='/op

In [3]:
import os
import gradio as gr
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv
import ollama

# Load environment variables from a .env file
load_dotenv()

# Set up constants for the Ollama API
OLLAMA_MODEL = "llama3.2-vision"  # Ensure this is the correct model name
BASE_URL = "http://localhost:11434"
TEMPERATURE = 0.7

# Nutrition analysis prompt
INPUT_PROMPT = """
You are an expert nutritionist. Analyze the food items in the provided image and calculate the total calories.
Provide details of each food item with its calorie intake in the following format:

1. Item 1 - [calories]
2. Item 2 - [calories]
----
"""

def process_uploaded_image(uploaded_image):
    """Convert the uploaded image to bytes and save it as a temporary file."""
    if uploaded_image is None:
        raise ValueError("No image uploaded")

    # Convert the PIL Image to bytes and save as a temporary file
    buffered = BytesIO()
    uploaded_image.save(buffered, format="JPEG")
    image_bytes = buffered.getvalue()
    
    # Save the image bytes to a temporary file
    image_path = "temp_image.jpg"
    with open(image_path, "wb") as f:
        f.write(image_bytes)
    
    return image_path

def get_ollama_response(user_input, image_path):
    """Generate a response using the Ollama LLM with image support."""
    try:
        # Use ollama.chat() method with image support
        response = ollama.chat(
            model=OLLAMA_MODEL,
            messages=[{
                'role': 'user',
                'content': f"{INPUT_PROMPT}\n{user_input}",
                'images': [image_path]
            }]
        )
        
        # Extract the text from the response
        if response and 'response' in response:
            result_text = response['response']
        else:
            result_text = 'No response received from the model.'
        
        print(f"Ollama Response: {result_text}")  # For debugging
        return result_text
    except Exception as e:
        return f"Error generating response: {e}"

def gemini_health_app(image, user_input):
    """Main function to process input and generate a response."""
    try:
        # Convert the uploaded image to a temporary file path
        image_path = process_uploaded_image(image)
        
        # Get the Ollama response with image and user input
        response = get_ollama_response(user_input, image_path)
        
        return response
    except Exception as e:
        return f"Error processing request: {e}"

# Create Gradio interface
iface = gr.Interface(
    fn=gemini_health_app,
    inputs=[
        gr.Image(type="pil", label="Upload Food Image"),
        gr.Textbox(label="Additional Input (Optional)", placeholder="E.g., Any dietary restrictions?")
    ],
    outputs=gr.Textbox(label="Calorie Analysis Result"),
    title="Gemini Health App",
    description="Upload an image of food items to get a detailed calorie count and nutrition analysis.",
    allow_flagging="never"
)

# Launch the Gradio interface
if __name__ == "__main__":
    iface.launch()


2024-11-17 15:47:10,924 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:10,925 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:10,926 [DEBUG] load_verify_locations cafile='/opt/anaconda3/envs/aienv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-11-17 15:47:10,944 [DEBUG] load_verify_locations cafile='/opt/anaconda3/envs/aienv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-11-17 15:47:10,982 [DEBUG] connect_tcp.started host='api.gradio.app' port=443 local_address=None timeout=3 socket_options=None
2024-11-17 15:47:10,982 [DEBUG] connect_tcp.started host='checkip.amazonaws.com' port=443 local_address=None timeout=3 socket_options=None
2024-11-17 15:47:11,033 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:11,033 [DEBUG] load_verify_locations cafile='/opt/anaconda3/envs/aienv/lib/python3.10/site-packages/certifi/cacert.pem'
2024-11-17 15:47:11,050 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2024-11-17 15:47:11,075 [DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
2024-11-17 15:47:11,076 [DEBUG] load_verify_locations cafile='/opt/anaconda3/envs/aienv/lib/python3.10/site-packages/certifi/cacert.pem'


In [1]:
from langchain_community.llms import Ollama
import logging
from PIL import Image
from io import BytesIO
import asyncio
import base64
import os
import time
import psutil  # For system resource monitoring (make sure to install this library)

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler()]
)

# Constants
MODEL_NAME = 'llama3.2-vision'
IMAGE_PATH = '/Users/vinitpahwa/Documents/GENAIProjects/YouTuberSummarizer/myfood.jpeg'
TIMEOUT = 90  # Further increased Timeout in seconds
MAX_SIZE = (400, 400)  # Reduce image size even more
JPEG_QUALITY = 30  # Further reduce quality for optimization
MAX_RETRIES = 3  # Number of retries

# Step 1: Optimize the Image
def optimize_image(image_path, max_size=MAX_SIZE):
    try:
        logging.info("Opening image from path: %s", image_path)
        img = Image.open(image_path)
        original_size = img.size
        logging.debug("Original image size: %s", original_size)

        # Resize the image
        img.thumbnail(max_size)
        optimized_path = "optimized_image.jpg"
        img.save(optimized_path, "JPEG", quality=JPEG_QUALITY)
        optimized_size = img.size
        logging.info("Image optimized and saved at: %s", optimized_path)
        logging.debug("Optimized image size: %s", optimized_size)
        return optimized_path
    except Exception as e:
        logging.error("Error optimizing image: %s", e)
        raise

# Monitor system resources
def log_system_resources():
    cpu_percent = psutil.cpu_percent()
    memory_info = psutil.virtual_memory()
    logging.info(f"CPU Usage: {cpu_percent}% | Available Memory: {memory_info.available / (1024 * 1024):.2f} MB")

# Step 2: Asynchronous Call to Ollama with retries
async def get_ollama_response_async(model, messages, timeout=TIMEOUT, retries=MAX_RETRIES):
    attempt = 1
    while attempt <= retries:
        try:
            logging.info(f"Attempt {attempt}: Sending request to Ollama...")
            log_system_resources()
            start_time = time.time()
            
            # Sending the async request
            response = await ollama.chat(
                model=model,
                messages=messages
            )
            
            end_time = time.time()
            logging.info("Ollama response received in %.2f seconds", end_time - start_time)
            
            if response:
                return response
            else:
                logging.warning("Received empty response from Ollama")
        
        except ollama.TimeoutError:
            logging.warning(f"Attempt {attempt}: Ollama response timed out after {timeout} seconds")
        except Exception as e:
            logging.error(f"Error during Ollama call: {e}")
        
        attempt += 1
        logging.info("Retrying...")
    
    return "Failed to get a response after multiple attempts."

# Main function to run the entire process
async def run_ollama_analysis(image_path):
    try:
        logging.info("Starting Gemini Health App analysis...")

        # Optimize the Image
        optimized_image_path = optimize_image(image_path)

        # Prepare messages for Ollama
        with open(optimized_image_path, "rb") as img_file:
            encoded_string = base64.b64encode(img_file.read()).decode('utf-8')

        messages = [{
            'role': 'user',
            'content': 'What is in this image?',
            'images': [encoded_string]
        }]
        logging.debug("Prepared messages for Ollama: %s", messages)

        # Get response from Ollama asynchronously
        response = await get_ollama_response_async(model=MODEL_NAME, messages=messages)

        logging.info("Response from Ollama: %s", response)
        return response

    except Exception as e:
        logging.error("Failed to run analysis: %s", e)
        return f"Error in analysis: {e}"

# Run the async event loop with compatibility for interactive environments
def main():
    if asyncio.get_event_loop().is_running():
        # If there's already an event loop running (e.g., in Jupyter), use it
        return asyncio.create_task(run_ollama_analysis(IMAGE_PATH))
    else:
        # Otherwise, run a new event loop
        asyncio.run(run_ollama_analysis(IMAGE_PATH))

if __name__ == "__main__":
    logging.info("Launching Gemini Health App...")
    main()


2024-11-17 15:52:21,440 [INFO] Launching Gemini Health App...
2024-11-17 15:52:21,444 [INFO] Starting Gemini Health App analysis...
2024-11-17 15:52:21,444 [INFO] Opening image from path: /Users/vinitpahwa/Documents/GENAIProjects/YouTuberSummarizer/myfood.jpeg


2024-11-17 15:52:21,456 [DEBUG] Original image size: (900, 1600)
2024-11-17 15:52:21,491 [INFO] Image optimized and saved at: optimized_image.jpg
2024-11-17 15:52:21,493 [DEBUG] Optimized image size: (225, 400)
2024-11-17 15:52:21,494 [DEBUG] Prepared messages for Ollama: [{'role': 'user', 'content': 'What is in this image?', 'images': ['/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDABsSFBcUERsXFhceHBsgKEIrKCUlKFE6PTBCYFVlZF9VXVtqeJmBanGQc1tdhbWGkJ6jq62rZ4C8ybqmx5moq6T/2wBDARweHigjKE4rK06kbl1upKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKSkpKT/wAARCAGQAOEDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRom